In [1]:
pip install faker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.7 MB/s eta 0:00:00


In [2]:
import sqlite3
import random
import string
from faker import Faker
from datetime import datetime

# Initialize Faker
fake = Faker()

# Connect to SQLite database
conn = sqlite3.connect("Shoe_Store_data.db")
cursor = conn.cursor()

# Create tables
cursor.executescript("""
    CREATE TABLE IF NOT EXISTS Customers (
        CustomerID INTEGER PRIMARY KEY AUTOINCREMENT,
        FirstName TEXT NOT NULL,
        LastName TEXT NOT NULL,
        Email TEXT UNIQUE NOT NULL,
        Phone TEXT CHECK (Phone IS NULL OR LENGTH(Phone) >= 10),
        Address TEXT,
        Age INTEGER CHECK (Age BETWEEN 18 AND 70),
        Gender TEXT CHECK (Gender IN ('Male', 'Female', 'Other'))
    );

    CREATE TABLE IF NOT EXISTS Shoes (
        ShoeID INTEGER PRIMARY KEY AUTOINCREMENT,
        Brand TEXT NOT NULL,
        Model TEXT NOT NULL,
        Category TEXT CHECK (Category IN ('Sneakers', 'Boots', 'Formal', 'Sandals', 'Sports')),
        Size INTEGER CHECK (Size BETWEEN 5 AND 15),
        Color TEXT,
        Price REAL CHECK (Price BETWEEN 50 AND 300)
    );

    CREATE TABLE IF NOT EXISTS Inventory (
        InventoryID INTEGER PRIMARY KEY AUTOINCREMENT,
        ShoeID INTEGER NOT NULL,
        Quantity INTEGER CHECK (Quantity >= 0),
        WarehouseLocation TEXT NOT NULL,
        FOREIGN KEY (ShoeID) REFERENCES Shoes(ShoeID)
    );

    CREATE TABLE IF NOT EXISTS Orders (
        OrderID INTEGER PRIMARY KEY AUTOINCREMENT,
        CustomerID INTEGER NOT NULL,
        OrderDate DATE DEFAULT (DATE('now')),
        TotalAmount REAL CHECK (TotalAmount >= 0),
        Status TEXT CHECK (Status IN ('Pending', 'Shipped', 'Delivered', 'Cancelled')),
        FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
    );

    CREATE TABLE IF NOT EXISTS OrderDetails (
        OrderID INTEGER NOT NULL,
        ShoeID INTEGER NOT NULL,
        Quantity INTEGER CHECK (Quantity > 0),
        PriceAtPurchase REAL CHECK (PriceAtPurchase > 0),
        PRIMARY KEY (OrderID, ShoeID),
        FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
        FOREIGN KEY (ShoeID) REFERENCES Shoes(ShoeID)
    );
""")


# Function to generate a truly unique email
def generate_email(existing_emails):

    """
    Generate a unique email address.
    Ensures that the email address is not already in the set of existing emails.

    Returns:
    - str: A unique email address.
    """

    while True:
        email = ''.join(random.choices(string.ascii_lowercase + string.digits, k=10)) + "@gmail.com"
        if email not in existing_emails:
            existing_emails.add(email)
            return email


# Function to generate a valid phone number with only digits and a '+' symbol
def generate_phone_number():

    """
    Generate a valid phone number consisting only of digits and a '+' symbol.
    The phone number will be between 10 and 12 digits.

    Returns:
    - str: A phone number starting with '+' followed by 10 to 12 digits.
    """
    return "+" + str(random.randint(1000000000, 999999999999))


# Insert customers (ensuring unique emails and some missing data)
def insert_customers(n=1000):
    """
    Insert `n` random customers into the Customers table.
    Ensures unique emails, some missing phone numbers, and some missing addresses.
    """
    customers = []
    unique_emails = set()

    for _ in range(n):
        email = generate_email(unique_emails)

        # 10% missing phone numbers, 5% missing addresses
        phone = generate_phone_number() if random.random() > 0.1 else None
        address = fake.address() if random.random() > 0.05 else None

        customers.append((
            fake.first_name(), fake.last_name(), email, phone, address,
            random.randint(18, 70), random.choice(['Male', 'Female', 'Other'])
        ))

    cursor.executemany("INSERT INTO Customers (FirstName, LastName, Email, Phone, Address, Age, Gender) VALUES (?, ?, ?, ?, ?, ?, ?)", customers)


# Insert shoes (with slight duplicates)
def insert_shoes(n=100):
    """
    Insert `n` random shoes into the Shoes table.
    Includes some duplicates with slight variations in model and price.

    Args:
    - n (int): The number of shoes to insert (default is 100).
    """
    brands = ["Nike", "Adidas", "Puma", "Reebok", "Vans", "New Balance"]
    categories = ["Sneakers", "Boots", "Formal", "Sandals", "Sports"]
    colors = ["Black", "White", "Red", "Blue", "Green", "Brown"]

    shoes = []
    for _ in range(n):
        shoe = (random.choice(brands), fake.word().capitalize(), random.choice(categories), random.randint(5, 15), random.choice(colors), round(random.uniform(50, 300), 2))
        shoes.append(shoe)

        if random.random() < 0.05:
            new_price = round(shoe[5] * 1.1, 2)
            new_price = min(max(new_price, 50), 300)
            shoes.append((shoe[0], shoe[1] + " V2", shoe[2], shoe[3], shoe[4], new_price))

    cursor.executemany("INSERT INTO Shoes (Brand, Model, Category, Size, Color, Price) VALUES (?, ?, ?, ?, ?, ?)", shoes)

# Insert inventory data
def insert_inventory():
    """
    Insert random inventory data into the Inventory table.
    This creates a random quantity of shoes and assigns a warehouse location to each.

    Args:
    - None
    """
    cursor.execute("SELECT ShoeID FROM Shoes")
    shoes = cursor.fetchall()

    inventory = [(shoe[0], random.randint(10, 500), fake.city()) for shoe in shoes]
    cursor.executemany("INSERT INTO Inventory (ShoeID, Quantity, WarehouseLocation) VALUES (?, ?, ?)", inventory)


# Insert orders allowing some duplicate customers
def insert_orders(n=1000):

    """
    Insert `n` random orders into the Orders table.
    Includes orders for existing customers and assigns random order statuses."""

    cursor.execute("SELECT CustomerID FROM Customers")
    customers = [c[0] for c in cursor.fetchall()]
    statuses = ["Pending", "Shipped", "Delivered", "Cancelled"]

    orders = []
    for _ in range(n):
        customer_id = random.choice(customers)

        if random.random() < 0.15:
            customer_id = random.choice(customers)

        order_date = fake.date_between(start_date="-2y", end_date="today").strftime('%Y-%m-%d')
        total_amount = round(random.uniform(50, 500), 2) if random.random() > 0.05 else None  # 5% NULL total amount
        status = random.choice(statuses)
        orders.append((customer_id, order_date, total_amount, status))
    cursor.executemany("INSERT INTO Orders (CustomerID, OrderDate, TotalAmount, Status) VALUES (?, ?, ?, ?)", orders)


# Insert order details (ensuring composite key constraints)
def insert_order_details():
    """
    Insert random order details into the OrderDetails table.
    Ensures no duplicate (OrderID, ShoeID) pairs by tracking previously inserted combinations.

    """
    cursor.execute("SELECT OrderID FROM Orders")
    orders = [o[0] for o in cursor.fetchall()]

    cursor.execute("SELECT ShoeID, Price FROM Shoes")
    shoes = cursor.fetchall()

    order_details = []
    seen_combinations = set()  # Track (OrderID, ShoeID) combinations

    for order_id in orders:
        shoe_id, price = random.choice(shoes)
        quantity = random.randint(1, 3)

        # Ensure no duplicate (OrderID, ShoeID) pair
        if (order_id, shoe_id) not in seen_combinations:
            order_details.append((order_id, shoe_id, quantity, price))
            seen_combinations.add((order_id, shoe_id))

    cursor.executemany("INSERT INTO OrderDetails (OrderID, ShoeID, Quantity, PriceAtPurchase) VALUES (?, ?, ?, ?)", order_details)


# Insert data
insert_customers(1000)
insert_shoes(100)
insert_inventory()
insert_orders(1000)
insert_order_details()

# Commit and close connection
conn.commit()
conn.close()

print("Shoe Store database created and data inserted with  meaningful constraints")


Shoe Store database created and data inserted with  meaningful constraints
